# Running a Qiskit Function as a function

In this tutorial, we will write a basic Qiskit Function using Qiskit Serverless. We will show how to run the function remotely and retrieve the results from the serverless client.

### Writing the Qiskit Function

First, we need to write the function code and save it to a file called [function.py](./source_files/function.py). This function creates a two-qubit quantum circuit that prepares a Bell state, measures the result, and saves the measured probability distribution.

The code for the function is shown below:

```python
from qiskit_serverless import save_result
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# all print statement will be available in job logs
print("Running function...")

# Set up a simulated backend and create a Bell state circuit.
backend = FakeAlmadenV2()
ideal_qc = QuantumCircuit(2)
ideal_qc.h(0)
ideal_qc.cx(0, 1)
ideal_qc.measure_all()

# Transpile the ideal circuit to one optimized for the backend.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_qc = pm.run(ideal_qc)

# Run the circuit on the sampler and get the measurement counts.
sampler = Sampler(mode=backend)
job = sampler.run([isa_qc], shots=1024)
result = job.result()[0].data.meas.get_counts()

# Save the final result for the serverless job.
save_result(result)
```

### Deploying the function

To run the function, we need to import the necessary classes and configure them. One of these classes is `ServerlessClient`, which is a client class for interacting with compute resources.

The client stores configuration information about our compute resources, such as where they are located and how to connect to them. In this example, we will use a provider that is connected to a local Docker Compose setup. In this case, it allows us to run the function locally on our machine. If you want to run the function elsewhere, you will need to provide the corresponding host and authentication details.

In [1]:
from qiskit_serverless import ServerlessClient
import os

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [2]:
client = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    # If you are using the kubernetes approach the URL must be http://localhost
)

client

<gateway-client>



`QiskitFunction` accepts couple of required parameters:

- title - name of the Qiskit Function
- entrypoint - name of python file you want to execute
- working_dir - directory where  your script is located (directory size must be less than 50MB). This is optional parameter and will be current folder by default.

> Warning! All content of `working_dir` will be shipped to cluster for execution

> Warning! Execution of `upload` function ships All content of `working_dir`.  When the contents of `working_dir` is changed, the `upload` function must be called again to update the shipped directory contents.

In [3]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="my-first-function", entrypoint="function.py", working_dir="./source_files/"
)

client.upload(function)

QiskitFunction(my-first-function)

### Running the QiskitFunction

After deploying the QiskitFunction, we can see our function in a `list` of availiable functions.

In [4]:
my_first_function = client.get("my-first-function")
my_first_function

QiskitFunction(my-first-function)

We can run any function by calling `run` method on function object.

In [5]:
job = my_first_function.run()
job

<Job | f22b56da-2ec8-49db-af32-f5cb729b3862>

[Job](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.Job.html#qiskit_serverless.core.Job) instances have a `status()` method to check status of the function execution.

In [6]:
job.status()

'QUEUED'

`Job` instances also have a `result()` method for retrieving results. The `result()` method will not return until the job is done running the function.

In [7]:

job.result()

{'00': 507, '11': 517}

To inspect the logs from a function, access them from the ``Job`` instance.

In [8]:
print(job.logs())

2025-08-12 07:06:09,369	INFO job_manager.py:531 -- Runtime env is setting up.
Running function...



`ServerlessClient` object has method `.widget` which renders Jupyter widget to see list of executed programs.

In [9]:
client.widget()